In [ ]:
from typing import *


from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
loader = DirectoryLoader(
    "../prompts/examples", glob="**/*.txt", loader_cls=TextLoader
)
docs = loader.load()
print(docs[0].page_content)

==== EXAMPLE APP DESCRIPTION ====

Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use the default port number. It should consume messages from 'receive_name' topic and the message will be a JSON encoded object with only one attribute: user_name. For each consumed message, construct a new message object and append 'Hello ' in front of the name attribute. Finally, publish the consumed message to 'send_greetings' topic.

==== EXAMPLE SKELETON CODE ====

from typing import *
from pydantic import BaseModel, Field
from fastkafka import FastKafka


class Greetings(BaseModel):
    user_name: str = Field(..., description="Name of the user.")

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "staging": {
        "url": "staging.airt.ai",
        "description": "staging kafka broker",
        "port": 9092,
    },

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "==== EXAMPLE SKELETON CODE ====",
    chunk_size = 2000,
    chunk_overlap  = 20
)

chunks = text_splitter.split_documents(docs)

chunks[0].page_content

"==== EXAMPLE APP DESCRIPTION ====\n\nCreate a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use the default port number. It should consume messages from 'receive_name' topic and the message will be a JSON encoded object with only one attribute: user_name. For each consumed message, construct a new message object and append 'Hello ' in front of the name attribute. Finally, publish the consumed message to 'send_greetings' topic."

In [ ]:
db = FAISS.from_documents(chunks, OpenAIEmbeddings()) # type: ignore
db.save_local("../tmp_db/faiss_index")

## Load db

In [ ]:
db = FAISS.load_local("../tmp_db/faiss_index", OpenAIEmbeddings()) # type: ignore

In [ ]:
query = """
Develop a FastKafka application using localhost broker for testing, staging.example.ai for staging and prod.example.ai for production. It should consume messages from 'course_updates' topic where the message is a JSON encoded object including two attributes: course_name and new_content. If new_content attribute is set, then construct a new message appending 'Updated: ' before the course_name attribute. Finally, publish this message to the 'notify_updates' topic. The application should use SASL_SSL with SCRAM-SHA-512 for authentication.
"""

results = db.max_marginal_relevance_search(query, k=3, fetch_k=4)
results

[Document(page_content="==== EXAMPLE APP DESCRIPTION ====\n\nCreate a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number. It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.\nFor each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic. Use SASL_SSL with SCRAM-SHA-256 for authentication with username and password.", metadata={'source': '../prompts/examples/example-1.txt'}),
 Document(page_content='==== EXAMPLE APP DESCRIPTION ====\n\nWrite a fastkafka application with with one consumer function and two producer functions. Th

In [ ]:
db.similarity_search("",filter=dict(source=results[0].metadata["source"]))

[Document(page_content='from typing import *\nfrom pydantic import BaseModel, Field\nfrom aiokafka.helpers import create_ssl_context\nfrom fastkafka import FastKafka\n\n\nclass StoreProduct(BaseModel):\n    product_name: str = Field(..., description="Name of the product.")\n    currency: str = Field(..., description="The currency.")\n    price: float = Field(..., description="Price of the product.")\n\n\nkafka_brokers = {\n    "localhost": {\n        "url": "localhost",\n        "description": "local development kafka broker",\n        "port": 9092,\n    },\n    "staging": {\n        "url": "staging.airt.ai",\n        "description": "staging kafka broker",\n        "port": 9092,\n        "protocol": "kafka-secure",\n        "security": {"type": "scramSha256"},\n    },\n    "production": {\n        "url": "prod.airt.ai",\n        "description": "production kafka broker",\n        "port": 9092,\n        "protocol": "kafka-secure",\n        "security": {"type": "scramSha256"},\n    }\n}\n

In [ ]:
full_result_docs = [db.similarity_search("",filter=dict(source=r.metadata["source"])) for r in results]
_input = full_result_docs[0][1].page_content
_output = full_result_docs[0][0].page_content.split("==== EXAMPLE APPLICATION CODE ====")[0]

In [ ]:
print(_input)

==== EXAMPLE APP DESCRIPTION ====

Create a FastKafka application using localhost broker for testing, staging.airt.ai for staging and prod.airt.ai for production. Use default port number. It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'.
For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic. Use SASL_SSL with SCRAM-SHA-256 for authentication with username and password.


In [ ]:
print(_output)

from typing import *
from pydantic import BaseModel, Field
from aiokafka.helpers import create_ssl_context
from fastkafka import FastKafka


class StoreProduct(BaseModel):
    product_name: str = Field(..., description="Name of the product.")
    currency: str = Field(..., description="The currency.")
    price: float = Field(..., description="Price of the product.")


kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "staging": {
        "url": "staging.airt.ai",
        "description": "staging kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "scramSha256"},
    },
    "production": {
        "url": "prod.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "scramSha256"},
    }
}

store_product_app_description = "A FastKafka application